In [1]:
import pandas as pd
import numpy as np

In [2]:
loc_lbl = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Labels"

In [3]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_auc"

In [4]:
auc_lb = pd.read_csv( loc_lbl + "\\auc_lb.csv", dtype = {"obs_18_20":"bool", "obs_21_23":"bool", "obs_24_26":"bool"} )

### Filtro de manera que tenga los datos necesarios

In [77]:
rh_18_20 = pd.read_csv( r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets\auc_18_20.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_18_20 = rh_18_20["ref_hash"].drop_duplicates().tolist()
rh_21_23 = pd.read_csv( r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets\auc_21_23.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_21_23 = rh_21_23["ref_hash"].drop_duplicates().tolist()

In [78]:
dt_18_20 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash"] ] #Datos del (18-20)
st_21_23 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_18_20 ), ["21_23_st"] ] #SC del (21-23)

dt_21_23 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash"] ] #Datos del (21-23)
st_24_26 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_21_23 ), ["24_26_st"] ] #SC del (24-26)

# <span style="color:orange">  Preparo los datos para predecir </span>
Con los datos 21-23 predecimos "24-26_sc"

In [79]:
Z = dt_21_23

### Agregamos los features 

In [80]:
ftr_01_Z = pd.read_csv( loc_ftr + "\\hora_auc_21_23.csv" ) 
ftr_02_Z = pd.read_csv( loc_ftr + "\\cant_clk_21_23.csv" ) 
ftr_03_Z = pd.read_csv( loc_ftr + "\\cant_ins_21_23.csv" ) 
ftr_04_Z = pd.read_csv( loc_ftr + "\\cant_auc_21_23.csv" ) 
ftr_05_Z = pd.read_csv( loc_ftr + "\\cant_evt_21_23.csv" )
ftr_06_Z = pd.read_csv( loc_ftr + "\\ref_type_21_23.csv" )  
ftr_07_Z = pd.read_csv( loc_ftr + "\\sdia_auc_21_23.csv" )
ftr_08_Z = pd.read_csv( loc_ftr + "\\srce_auc_21_23.csv" )
ftr_09_Z = pd.read_csv( loc_ftr + "\\rh_encod_21_23.csv" )
ftr_10_Z = pd.read_csv( loc_ftr + "\\main_ahr_21_23.csv" )

In [81]:
Z = Z.merge( ftr_01_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_02_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_03_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_04_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_05_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_06_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_07_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_08_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_09_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_10_Z, how = "inner", on = "ref_hash" )

# <span style="color:orange">  Preparo los datos para entrenar </span>

Con los datos 18-20 + "21-23_sc" entrenamos

In [82]:
Y = st_21_23

In [83]:
X = dt_18_20

### Agregamos los features

In [70]:
ftr_01_X = pd.read_csv( loc_ftr + "\\hora_auc_18_20.csv" ) 
ftr_02_X = pd.read_csv( loc_ftr + "\\cant_clk_18_20.csv" ) 
ftr_03_X = pd.read_csv( loc_ftr + "\\cant_ins_18_20.csv" ) 
ftr_04_X = pd.read_csv( loc_ftr + "\\cant_auc_18_20.csv" ) 
ftr_05_X = pd.read_csv( loc_ftr + "\\cant_evt_18_20.csv" )
ftr_06_X = pd.read_csv( loc_ftr + "\\ref_type_18_20.csv" )  
ftr_07_X = pd.read_csv( loc_ftr + "\\sdia_auc_18_20.csv" )  
ftr_08_X = pd.read_csv( loc_ftr + "\\srce_auc_18_20.csv" )
ftr_09_X = pd.read_csv( loc_ftr + "\\rh_encod_18_20.csv" )
ftr_10_X = pd.read_csv( loc_ftr + "\\main_ahr_18_20.csv" )

In [71]:
X = X.merge( ftr_01_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_02_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_03_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_04_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_05_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_06_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_07_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_08_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_09_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_10_X, how = "inner", on = "ref_hash" )

## Dropeamos los ref_hash (Quedan los codificados del feature)

In [72]:
X = X.drop( "ref_hash", axis = 1 )
Z = Z.drop( "ref_hash", axis = 1 )

# Prediccion con <span style="color:green"> *XGBoost*</span> 

In [73]:
import xgboost as xgb
import graphviz as gr 

In [74]:
prm = {"objective" :'reg:linear', 
       "colsample_bytree" : 0.3, 
       "learning_rate" :0.1,
       "max_depth" : 4, 
       "alpha" : 10, 
       "n_estimators" : 150 }

estimator = xgb.XGBRegressor( **prm )

In [75]:
pmt = estimator.fit( X,Y )

In [76]:
prd = estimator.predict( Z )

### Calculamos el RMSE

In [58]:
from sklearn.metrics import mean_squared_error

In [59]:
rmse = np.sqrt( mean_squared_error(st_24_26, prd) )
print("RMSE: %f" % (rmse) )

RMSE: 94907.152290


In [61]:
record = 94907.152290

## Graficamos el Arbol

In [ ]:
import matplotlib.pyplot as plt

import os; os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

dtrn = xgb.DMatrix(X,Y)

xg_reg = xgb.train(params = prm, dtrain = dtrn, num_boost_round=10)

xgb.plot_tree(xg_reg,num_trees=2)
plt.rcParams['figure.figsize'] = [25, 15]
plt.show()